In [1]:
import numpy as np
np.random.seed(2020)

import os
import glob
import cv2
import math
import pickle
import datetime
import pandas as pd
import statistics
import random
import time
import tensorflow.compat.v1 as tf
import keras

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint, EarlyStopping

from sklearn.metrics import log_loss
from imageio import imread
# Use numpy.array(Image.fromarray(arr).resize()) for image resizing if we need it

sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
tf.compat.v1.keras.backend.set_session(sess)

#globals for rows and cols since we will always be doing images of the same size and they will always be 3 for RGB or 1 for Gray
rows = 80
cols = 60
RGB = 1


Using TensorFlow backend.


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device



In [2]:
def storeData(data, fPath):
    f = open(fPath, 'wb')
    pickle.dump(data, f, protocol=4)
    f.close()

def getData(fPath):
    if os.path.isfile(fPath):
        f = open(fPath, 'rb')
        data = pickle.load(f)
    else:
        print('File does not exist')
        print('Returning empty dict instead')
        data = dict()
    return data

def get_driver_data():
    dr = dict()
    path = ('driver_imgs_list.csv')
    #print(path)
    print('Read drivers data')
    f = open(path, 'r')
    line = f.readline()
    while (1):
        line = f.readline()
        if line == '':
            break
        arr = line.strip().split(',')
        dr[arr[2]] = arr[0]
    f.close()
    return dr

def load_train():
    trainFile = 'train.pickle'
    if os.path.isfile(trainFile):
        X_train, y_train, driver_id, unique_drivers = getData(trainFile)
        print(unique_drivers)
        return X_train, y_train, driver_id, unique_drivers
    else:
        X_train, y_train, driver_id, unique_drivers = train_helper()
        storeData((X_train, y_train, driver_id, unique_drivers),trainFile)
        return X_train, y_train, driver_id, unique_drivers

def train_helper():
    X_train = []
    y_train = []
    driver_id = []
    start_time = time.time()
    driver_data = get_driver_data()

    print('Read train images')
    for j in range(10):
        print('Load folder c{}'.format(j))
        path = os.path.join('train', 'c' + str(j), '*.jpg')
        #print(path)
        #return
        files = glob.glob(path)
        for fl in files:
            if RGB == 1:
                img = cv2.imread(fl, 0)
            else:
                img = cv2.imread(fl)
            #img = get_im_cv2_mod(fl, img_rows, img_cols, color_type)
            X_train.append(img)
            y_train.append(j)
            driver_id.append(driver_data[os.path.basename(fl)])

    print('Read train data time: {} seconds'.format(round(time.time() - start_time, 2)))
    unique_drivers = sorted(list(set(driver_id)))
    print('Unique drivers: {}'.format(len(unique_drivers)))
    #print(unique_drivers)
    
    X_train = np.array(X_train, dtype=np.uint8)
    y_train = np.array(y_train, dtype=np.uint8)
    
    X_train = X_train.reshape(X_train.shape[0], RGB, rows, cols)
    y_train = np_utils.to_categorical(y_train, 10)
    
    X_train = X_train.astype('float32')
    X_train /= 255
    
    return X_train, y_train, driver_id, unique_drivers

def load_test():
    testFile = 'test.pickle'
    if os.path.isfile(testFile):
        X_test, X_test_id = getData(testFile)
        return X_test, X_test_id
    else:
        X_test, X_test_id = test_helper()
        storeData((X_test, X_test_id),testFile)
        return X_test, X_test_id

def test_helper():
    print('Read test images')
    start_time = time.time()
    path = os.path.join('test', '*.jpg')
    files = glob.glob(path)
    X_test = []
    X_test_id = []
    total = 0
    thr = math.floor(len(files)/10)
    for fl in files:
        if RGB == 1:
            img = cv2.imread(fl, 0)
        else:
            img = cv2.imread(fl)
        X_test.append(img)
        X_test_id.append(os.path.basename(fl))
        total += 1
        if total%thr == 0:
            print('Read {} images from {}'.format(total, len(files)))
    
    print('Read test data time: {} seconds'.format(round(time.time() - start_time, 2)))
    
    X_test = np.array(X_test, dtype=np.uint8)
    X_test = X_test.reshape(X_test.shape[0], RGB, rows, cols)
    
    X_test = X_test.astype('float32')
    X_test /= 255
    
    return X_test, X_test_id

def copy_selected_drivers(train_data, train_target, driver_id, driver_list):
    data = []
    target = []
    index = []
    for i in range(len(driver_id)):
        if driver_id[i] in driver_list:
            data.append(train_data[i])
            target.append(train_target[i])
            index.append(i)
    data = np.array(data, dtype=np.float32)
    target = np.array(target, dtype=np.float32)
    index = np.array(index, dtype=np.uint32)
    return data, target, index

def create_model():
    model = Sequential()
    model.add(Convolution2D(32, (3, 3), input_shape=(RGB, rows, cols), padding="same", kernel_initializer="he_normal"))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))
    model.add(Dropout(0.5))

    model.add(Convolution2D(32, (3, 3), padding="same", kernel_initializer="he_normal"))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))
    model.add(Dropout(0.5))

    model.add(Convolution2D(128, (3, 3), padding="same", kernel_initializer="he_normal"))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_model2():
    model = Sequential()
    model.add(Convolution2D(32, (3, 3), input_shape=(RGB, rows, cols), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Convolution2D(64, (3, 3), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Convolution2D(128, (3, 3), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Flatten())
    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_model3():
    model = Sequential()
    model.add(Convolution2D(32, (3, 3), input_shape=(RGB, rows, cols), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Convolution2D(64, (3, 3), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Convolution2D(128, (3, 3), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Flatten())
    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(optimizer='rmsprop', loss='kullback_leibler_divergence', metrics=['accuracy'])
    return model

def create_model4():
    model = Sequential()
    model.add(Convolution2D(32, (3, 3), input_shape=(RGB, rows, cols), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Convolution2D(64, (3, 3), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Convolution2D(128, (3, 3), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Flatten())
    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(optimizer='adam', loss='kullback_leibler_divergence', metrics=['accuracy'])
    return model

def create_model5():
    model = Sequential()

    ## CNN 1
    model.add(Convolution2D(32,(3,3),activation='relu',input_shape=(RGB, rows, cols)))
    model.add(BatchNormalization())
    model.add(Convolution2D(32,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization(axis = 3))
    model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
    model.add(Dropout(0.3))

    ## CNN 2
    model.add(Convolution2D(64,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Convolution2D(64,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization(axis = 3))
    model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
    model.add(Dropout(0.3))

    ## CNN 3
    model.add(Convolution2D(128,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Convolution2D(128,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization(axis = 3))
    model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
    model.add(Dropout(0.5))

    ## Output
    model.add(Flatten())
    model.add(Dense(512,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(10,activation='softmax'))

    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_model6():
    model = Sequential()
    model.add(Convolution2D(32, (3, 3), input_shape=(RGB, rows, cols), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(AveragePooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Convolution2D(64, (3, 3), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(AveragePooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Convolution2D(128, (3, 3), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(AveragePooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Flatten())
    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(optimizer='rmsprop', loss='kullback_leibler_divergence', metrics=['accuracy'])
    return model

def create_model7():
    model = Sequential()
    model.add(Convolution2D(32, (3, 3), input_shape=(RGB, rows, cols), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(AveragePooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Convolution2D(64, (3, 3), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(AveragePooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Convolution2D(128, (3, 3), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(AveragePooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Flatten())
    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(optimizer='adam', loss='kullback_leibler_divergence', metrics=['accuracy'])
    return model


In [3]:
train_data, train_target, driver_id, unique_drivers = load_train()
test_data, test_id = load_test()
# train_datagen = ImageDataGenerator(rescale = 1.0/255, 
#                                    shear_range = 0.2, 
#                                    zoom_range = 0.2, 
#                                    horizontal_flip = True, 
#                                    validation_split = 0.2)

# training_set = train_datagen.flow_from_directory('train', 
#                                                  target_size = (240, 240), 
#                                                  batch_size = 32,
#                                                  subset = 'training')

# validation_set = train_datagen.flow_from_directory('train', 
#                                                    target_size = (240, 240), 
#                                                    batch_size = 32,
#                                                    subset = 'validation')

Read drivers data
Read train images
Load folder c0
Load folder c1
Load folder c2
Load folder c3
Load folder c4
Load folder c5
Load folder c6
Load folder c7
Load folder c8
Load folder c9
Read train data time: 4.04 seconds
Unique drivers: 26
Read test images
Read 7972 images from 79726
Read 15944 images from 79726
Read 23916 images from 79726
Read 31888 images from 79726
Read 39860 images from 79726
Read 47832 images from 79726
Read 55804 images from 79726
Read 63776 images from 79726
Read 71748 images from 79726
Read 79720 images from 79726
Read test data time: 237.28 seconds


In [4]:
#########RMS and Cross Entrop

batch_size = 32
epochs = 10

yfull_train = dict()
yfull_test = []


unique_list_train = ['p002', 'p012', 'p014', 'p015', 'p016', 'p021', 'p022', 'p024',
                     'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049',
                     'p050', 'p051', 'p052', 'p056', 'p061', 'p064', 'p066', 'p072',
                     'p075']
X_train, Y_train, train_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_train)
unique_list_valid = ['p081']
X_valid, Y_valid, test_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_valid)

print('Start Single Run')
print('Split train: ', len(X_train), len(Y_train))
print('Split valid: ', len(X_valid), len(Y_valid))
print('Train drivers: ', unique_list_train)
print('Test drivers: ', unique_list_valid)


model = create_model2()
model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs,
              verbose=1, validation_data=(X_valid, Y_valid))

# score = model.evaluate(X_valid, Y_valid, show_accuracy=True, verbose=0)
# print('Score log_loss: ', score[0])

predictions_valid = model.predict(X_valid, batch_size=128, verbose=1)
score = log_loss(Y_valid, predictions_valid)
print('Score log_loss: ', score)

# Store valid predictions
for i in range(len(test_index)):
    yfull_train[test_index[i]] = predictions_valid[i]

# Store test predictions
test_prediction = model.predict(test_data, batch_size=128, verbose=1)
yfull_test.append(test_prediction)

print('Final log_loss: {}, rows: {} cols: {} epochs: {}'.format(score, rows, cols, epochs))
info_string = 'loss_' + str(score) \
                + '_r_' + str(rows) \
                + '_c_' + str(cols) \
                + '_ep_' + str(epochs)

Start Single Run
Split train:  21601 21601
Split valid:  823 823
Train drivers:  ['p002', 'p012', 'p014', 'p015', 'p016', 'p021', 'p022', 'p024', 'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049', 'p050', 'p051', 'p052', 'p056', 'p061', 'p064', 'p066', 'p072', 'p075']
Test drivers:  ['p081']
Train on 21601 samples, validate on 823 samples
Epoch 1/10
21601/21601 [==============================] - 6s 289us/step - loss: 2.4912 - accuracy: 0.1770 - val_loss: 2.4486 - val_accuracy: 0.0948
Epoch 2/10
21601/21601 [==============================] - 3s 152us/step - loss: 1.7908 - accuracy: 0.3570 - val_loss: 2.5901 - val_accuracy: 0.0778
Epoch 3/10
21601/21601 [==============================] - 3s 154us/step - loss: 1.4212 - accuracy: 0.4967 - val_loss: 2.4239 - val_accuracy: 0.0996
Epoch 4/10
21601/21601 [==============================] - 3s 155us/step - loss: 1.1704 - accuracy: 0.5941 - val_loss: 2.7041 - val_accuracy: 0.0620
Epoch 5/10
21601/21601 [==============================

In [ ]:
############RMS AND KL DIV###########


batch_size = 32
epochs = 10

yfull_train = dict()
yfull_test = []


unique_list_train = ['p002', 'p012', 'p014', 'p015', 'p016', 'p021', 'p022', 'p024',
                     'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049',
                     'p050', 'p051', 'p052', 'p056', 'p061', 'p064', 'p066', 'p072',
                     'p075']
X_train, Y_train, train_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_train)
unique_list_valid = ['p081']
X_valid, Y_valid, test_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_valid)

print('Start Single Run')
print('Split train: ', len(X_train), len(Y_train))
print('Split valid: ', len(X_valid), len(Y_valid))
print('Train drivers: ', unique_list_train)
print('Test drivers: ', unique_list_valid)


model = create_model3()
model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs,
              verbose=1, validation_data=(X_valid, Y_valid))

# score = model.evaluate(X_valid, Y_valid, show_accuracy=True, verbose=0)
# print('Score log_loss: ', score[0])

predictions_valid = model.predict(X_valid, batch_size=128, verbose=1)
score = log_loss(Y_valid, predictions_valid)
print('Score log_loss: ', score)

# Store valid predictions
for i in range(len(test_index)):
    yfull_train[test_index[i]] = predictions_valid[i]

# Store test predictions
test_prediction = model.predict(test_data, batch_size=128, verbose=1)
yfull_test.append(test_prediction)

print('Final log_loss: {}, rows: {} cols: {} epochs: {}'.format(score, rows, cols, epochs))
info_string = 'loss_' + str(score) \
                + '_r_' + str(rows) \
                + '_c_' + str(cols) \
                + '_ep_' + str(epochs)

In [ ]:
############ADAM AND KL DIV###########


batch_size = 32
epochs = 10

yfull_train = dict()
yfull_test = []


unique_list_train = ['p002', 'p012', 'p014', 'p015', 'p016', 'p021', 'p022', 'p024',
                     'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049',
                     'p050', 'p051', 'p052', 'p056', 'p061', 'p064', 'p066', 'p072',
                     'p075']
X_train, Y_train, train_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_train)
unique_list_valid = ['p081']
X_valid, Y_valid, test_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_valid)

print('Start Single Run')
print('Split train: ', len(X_train), len(Y_train))
print('Split valid: ', len(X_valid), len(Y_valid))
print('Train drivers: ', unique_list_train)
print('Test drivers: ', unique_list_valid)


model = create_model4()
model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs,
              verbose=1, validation_data=(X_valid, Y_valid))

# score = model.evaluate(X_valid, Y_valid, show_accuracy=True, verbose=0)
# print('Score log_loss: ', score[0])

predictions_valid = model.predict(X_valid, batch_size=128, verbose=1)
score = log_loss(Y_valid, predictions_valid)
print('Score log_loss: ', score)

# Store valid predictions
for i in range(len(test_index)):
    yfull_train[test_index[i]] = predictions_valid[i]

# Store test predictions
test_prediction = model.predict(test_data, batch_size=128, verbose=1)
yfull_test.append(test_prediction)

print('Final log_loss: {}, rows: {} cols: {} epochs: {}'.format(score, rows, cols, epochs))
info_string = 'loss_' + str(score) \
                + '_r_' + str(rows) \
                + '_c_' + str(cols) \
                + '_ep_' + str(epochs)

In [ ]:
############rms, crossentropy and softmax###########


batch_size = 32
epochs = 10

yfull_train = dict()
yfull_test = []


unique_list_train = ['p002', 'p012', 'p014', 'p015', 'p016', 'p021', 'p022', 'p024',
                     'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049',
                     'p050', 'p051', 'p052', 'p056', 'p061', 'p064', 'p066', 'p072',
                     'p075']
X_train, Y_train, train_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_train)
unique_list_valid = ['p081']
X_valid, Y_valid, test_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_valid)

print('Start Single Run')
print('Split train: ', len(X_train), len(Y_train))
print('Split valid: ', len(X_valid), len(Y_valid))
print('Train drivers: ', unique_list_train)
print('Test drivers: ', unique_list_valid)


model = create_model5()
model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs,
              verbose=1, validation_data=(X_valid, Y_valid))

# score = model.evaluate(X_valid, Y_valid, show_accuracy=True, verbose=0)
# print('Score log_loss: ', score[0])

predictions_valid = model.predict(X_valid, batch_size=128, verbose=1)
score = log_loss(Y_valid, predictions_valid)
print('Score log_loss: ', score)

# Store valid predictions
for i in range(len(test_index)):
    yfull_train[test_index[i]] = predictions_valid[i]

# Store test predictions
test_prediction = model.predict(test_data, batch_size=128, verbose=1)
yfull_test.append(test_prediction)

print('Final log_loss: {}, rows: {} cols: {} epochs: {}'.format(score, rows, cols, epochs))
info_string = 'loss_' + str(score) \
                + '_r_' + str(rows) \
                + '_c_' + str(cols) \
                + '_ep_' + str(epochs)

In [ ]:
############rms, kldiv and softmax###########


batch_size = 32
epochs = 10

yfull_train = dict()
yfull_test = []


unique_list_train = ['p002', 'p012', 'p014', 'p015', 'p016', 'p021', 'p022', 'p024',
                     'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049',
                     'p050', 'p051', 'p052', 'p056', 'p061', 'p064', 'p066', 'p072',
                     'p075']
X_train, Y_train, train_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_train)
unique_list_valid = ['p081']
X_valid, Y_valid, test_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_valid)

print('Start Single Run')
print('Split train: ', len(X_train), len(Y_train))
print('Split valid: ', len(X_valid), len(Y_valid))
print('Train drivers: ', unique_list_train)
print('Test drivers: ', unique_list_valid)


model = create_model6()
model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs,
              verbose=1, validation_data=(X_valid, Y_valid))

# score = model.evaluate(X_valid, Y_valid, show_accuracy=True, verbose=0)
# print('Score log_loss: ', score[0])

predictions_valid = model.predict(X_valid, batch_size=128, verbose=1)
score = log_loss(Y_valid, predictions_valid)
print('Score log_loss: ', score)

# Store valid predictions
for i in range(len(test_index)):
    yfull_train[test_index[i]] = predictions_valid[i]

# Store test predictions
test_prediction = model.predict(test_data, batch_size=128, verbose=1)
yfull_test.append(test_prediction)

print('Final log_loss: {}, rows: {} cols: {} epochs: {}'.format(score, rows, cols, epochs))
info_string = 'loss_' + str(score) \
                + '_r_' + str(rows) \
                + '_c_' + str(cols) \
                + '_ep_' + str(epochs)

In [ ]:
############adam, kldiv and softmax###########


batch_size = 32
epochs = 10

yfull_train = dict()
yfull_test = []


unique_list_train = ['p002', 'p012', 'p014', 'p015', 'p016', 'p021', 'p022', 'p024',
                     'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049',
                     'p050', 'p051', 'p052', 'p056', 'p061', 'p064', 'p066', 'p072',
                     'p075']
X_train, Y_train, train_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_train)
unique_list_valid = ['p081']
X_valid, Y_valid, test_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_valid)

print('Start Single Run')
print('Split train: ', len(X_train), len(Y_train))
print('Split valid: ', len(X_valid), len(Y_valid))
print('Train drivers: ', unique_list_train)
print('Test drivers: ', unique_list_valid)


model = create_model7()
model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs,
              verbose=1, validation_data=(X_valid, Y_valid))

# score = model.evaluate(X_valid, Y_valid, show_accuracy=True, verbose=0)
# print('Score log_loss: ', score[0])

predictions_valid = model.predict(X_valid, batch_size=128, verbose=1)
score = log_loss(Y_valid, predictions_valid)
print('Score log_loss: ', score)

# Store valid predictions
for i in range(len(test_index)):
    yfull_train[test_index[i]] = predictions_valid[i]

# Store test predictions
test_prediction = model.predict(test_data, batch_size=128, verbose=1)
yfull_test.append(test_prediction)

print('Final log_loss: {}, rows: {} cols: {} epochs: {}'.format(score, rows, cols, epochs))
info_string = 'loss_' + str(score) \
                + '_r_' + str(rows) \
                + '_c_' + str(cols) \
                + '_ep_' + str(epochs)